# NYC Bus Delays

[This dataset](https://www.kaggle.com/datasets/stoney71/new-york-city-transport-statistics) is from the New York City Metropolitan Transit Authority (NYC MTA) buses data stream service. In roughly 10 minute increments the bus location, route, next stop and more is recorded. Using this data, I aim to identify where buses tend to pick up delays, in order to propose where additional journey time should be built into the timetable to increase punctuality performance.

Here I have used the 1706 dataset, but the code should work equally well for any of the other datasets from the group, or (with enough running time) for the compiled datasets.

## Importing and cleaning the data

In [57]:
import pandas as pd

# Credit to https://www.kaggle.com/code/arosebudrejoicing/late-new-york-buses for inspiring the method of dealing with 
# unexpected columns in the dataset, caused by a stray comma which has caused some data to stray out of the intended columns

# Load the csv with numbers instead of column headers
names=range(19)
df_1706 = pd.read_csv("./mta_1706.csv", header=None, names=names, low_memory=False)

# Get the correct column names back
df_1706.columns = pd.concat([df_1706.iloc[0, :-2], pd.Series([17,18])])
df_1706 = df_1706.drop(0, axis=0)
df_1706

# Just use a sample of the whole dataset for testing purposes, otherwise this will take a long time to run on the 
# millions of rows in the full dataset
df_1706 = df_1706.sample(n=30000)

In [58]:
# Check what the extra two columns contain
df_1706[[17, 18]].describe(include="all")

,17,18
count,1,0.0
unique,1,NaN
top,15:46:08,NaN
freq,1,NaN
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN


In [59]:
# Column 18 is empty, so we can remove it
df_1706 = df_1706.drop(18, axis=1)

In [60]:
# Column 17 contains data which has accidentally been nudged over from columns to the left
# We can get a DataFrame of the rows which are affected by this issue
df_1706_error_rows = df_1706.loc[~df_1706[17].isna()].copy()
df_1706_error_rows

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,17
3034337,2017-08-14 15:56:17,0,M5,6 AV/W 31 ST,40.748043,-73.98896,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848263,-73.937454,NYCT_3853,40.776116,-73.982017,W 72 ST/Amsterdam av ( non-public,for GEO),< 1 stop away,299,2017-08-14 15:58:02,15:46:08


In [61]:
# We can see that to fix these rows, we need to concatenate NextStopPointName and 
# ArrivalProximityText, then drop the ArrivalProximityText column
df_1706_error_rows.loc[:, "NextStopPointName"] = df_1706_error_rows.loc[:, "NextStopPointName"] + " " + df_1706_error_rows.loc[:, "ArrivalProximityText"]
df_1706_error_rows = df_1706_error_rows.drop("ArrivalProximityText", axis=1)

# ... and then rename the columns from 'DistanceFromStop' onwards
df_1706_error_rows = df_1706_error_rows.rename(columns={"DistanceFromStop":"ArrivalProximityText", "ExpectedArrivalTime":"DistanceFromStop", "ScheduledArrivalTime":"ExpectedArrivalTime",17:"ScheduledArrivalTime"})

In [62]:
# We can now insert the corrected rows back into the main DataFrame and drop column 17 entirely
df_1706.update(df_1706_error_rows, overwrite=True)
df_1706 = df_1706.drop(17, axis=1)

In [63]:
# We can now have a look and see where there are missing values; there is still some cleaning-up to do
df_1706.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 4615929 to 111121
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   RecordedAtTime             30000 non-null  object
 1   DirectionRef               30000 non-null  object
 2   PublishedLineName          30000 non-null  object
 3   OriginName                 29621 non-null  object
 4   OriginLat                  29621 non-null  object
 5   OriginLong                 29621 non-null  object
 6   DestinationName            30000 non-null  object
 7   DestinationLat             29943 non-null  object
 8   DestinationLong            29943 non-null  object
 9   VehicleRef                 30000 non-null  object
 10  VehicleLocation.Latitude   30000 non-null  object
 11  VehicleLocation.Longitude  30000 non-null  object
 12  NextStopPointName          29971 non-null  object
 13  ArrivalProximityText       29971 non-null  object
 14 

In [64]:
# We need to convert all the location data into numerical datatypes
df_1706["OriginLat"] = pd.to_numeric(df_1706["OriginLat"], errors='raise')
df_1706["OriginLong"] = pd.to_numeric(df_1706["OriginLong"], errors='raise')
df_1706["DestinationLat"] = pd.to_numeric(df_1706["DestinationLat"], errors='raise')
df_1706["DestinationLong"] = pd.to_numeric(df_1706["DestinationLong"], errors='raise')
df_1706["VehicleLocation.Latitude"] = pd.to_numeric(df_1706["VehicleLocation.Latitude"], errors='raise')
df_1706["VehicleLocation.Longitude"] = pd.to_numeric(df_1706["VehicleLocation.Longitude"], errors='raise')

In [65]:
# We need to get separate the 'time' component from the expected arrival datetimes
df_1706["ExpectedArrivalTime"] = df_1706["ExpectedArrivalTime"].str.split(" ", expand=True).iloc[:, 1]

# We can then drop any rows with missing values; those missing values are mostly in the ExpectedArrivalTime column, and since 
# that's mainly what we're looking at, if that information is missing the whole row is of no use
df_1706 = df_1706.dropna()

# There's also an extraneous 'index' column which we can just get rid of
df_1706 = df_1706.reset_index().drop("index", axis=1)

In [66]:
# Now we convert the scheduled and expected arrival times into seconds, and in the process deal with the fact that in some cases
# times past midnight are represented as 00:01 and sometimes as 24:01, and we need a way to deal with scheduled/expected arrival 
# times that are on different dates; we deal with that by assuming that no bus will ever be more than an hour early

def time_string_to_seconds(time):
    if time == "nan":
        return "NaN"
    t = list(time.split(":"))
    total = int(t[0]) * 3600 + (0 if len(t) < 2 else int(t[1]) * 60) + (0 if len(t) < 3 else int(t[2]))
    if total > 86400:
        return (total - 86400)
    else:
        return total

df_1706["ScheduledArrivalTime_seconds"] = df_1706["ScheduledArrivalTime"].apply(lambda x: time_string_to_seconds(str(x)))
df_1706["ExpectedArrivalTime_seconds"] = df_1706["ExpectedArrivalTime"].apply(lambda x: time_string_to_seconds(str(x)))
df_1706["ExpectedArrivalTime_seconds"] = df_1706["ExpectedArrivalTime_seconds"].fillna(df_1706["ScheduledArrivalTime_seconds"])

df_1706["ExpectedDelay_seconds"] = df_1706["ExpectedArrivalTime_seconds"] - df_1706["ScheduledArrivalTime_seconds"]
df_1706["ExpectedDelay_seconds"] = df_1706["ExpectedDelay_seconds"].apply(lambda x: (x + 86400) if (x < -3600) else x)
df_1706["ExpectedDelay_seconds"] = df_1706["ExpectedDelay_seconds"].apply(lambda x: x if (x > 0) else 0)

In [67]:
df_1706.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,ScheduledArrivalTime_seconds,ExpectedArrivalTime_seconds,ExpectedDelay_seconds
0,2017-08-21 17:18:50,1,Q88,AMBOY LA/SPRINGFIELD BL,40.717834,-73.735626,WOODHVN BL STA QNS CTR MALL via H.HRDNG,40.734978,-73.869591,NYCT_8501,40.735035,-73.869523,92 ST/59 AV,at stop,4,17:19:04,17:12:00,61920,62344,424
1,2017-08-23 20:12:18,0,M103,PARK ROW/BEEKMAN ST,40.711861,-74.006157,E HARLEM 125 ST via 3 AV,40.803875,-73.933876,NYCT_6059,40.729048,-73.990272,3 AV/E 9 ST,< 1 stop away,172,20:13:21,20:11:17,72677,72801,124
2,2017-08-21 08:48:18,1,B82,SEAVIEW AV/PENNSYLVANIA AV,40.642979,-73.878334,CONEY ISLAND via FLATLANDS via KINGS HWY,40.576973,-73.981400,NYCT_6807,40.616386,-73.944430,KINGS HY/NOSTRAND AV,at stop,27,08:48:25,08:33:24,30804,31705,901
3,2017-08-04 09:28:24,0,Bx41-SBS,3 AVE/WESTCHESTER AV,40.816803,-73.916328,SELECT BUS WLIMSBRIDGE GUN HILL RD,40.877762,-73.865990,NYCT_5774,40.818219,-73.916793,MELROSE AV/E 161 ST,< 1 stop away,746,09:32:28,09:30:22,34222,34348,126
4,2017-08-17 13:41:22,0,B44-SBS,KNAPP ST/SHORE PKY,40.585033,-73.931328,SBS WLMSBRG BRDG PLZ via NSTRND via RGRS,40.709316,-73.959587,NYCT_5360,40.654518,-73.952882,ROGERS AV/CLARKSON AV,approaching,111,13:41:54,13:45:36,49536,49314,0


## Investigating delays

In [68]:
# How can we work out where on a route a delay is picked up?
# For a given VehicleRef, we can find when the ExpectedDelay_seconds increases (by, for each row, finding the previous check-in 
# of that vehicle,if it's within the last 500 rows of data, and comparing the ExpectedDelay_seconds values)
# Then we get the vehicle's location at each of those check-ins, and we know the delay was created between those locations

for i in df_1706.index:
    if (df_1706.iloc[i]["ExpectedDelay_seconds"] > 30) & (i > df_1706.index[0]):
        maxlookback = 500
        if i < maxlookback:
            lookback = i
        else:
            lookback = maxlookback
        lookback_df = df_1706.iloc[i-lookback:i].loc[df_1706["VehicleRef"] == df_1706.iloc[i]["VehicleRef"]]
        if len(lookback_df) > 0:
            prev_ind = lookback_df.iloc[-1].name
            df_1706.loc[i, "DelayIncreaseSincePrevCheckin"] = df_1706.iloc[i]["ExpectedDelay_seconds"] - df_1706.iloc[prev_ind]["ExpectedDelay_seconds"]
            df_1706.loc[i, "PrevCheckin_Latitude"] = df_1706.iloc[prev_ind]["VehicleLocation.Latitude"]
            df_1706.loc[i, "PrevCheckin_Longitude"] = df_1706.iloc[prev_ind]["VehicleLocation.Longitude"]

In [69]:
# Now we create a separate DataFrame containing just the information about where delays were introduced
df_1706_delay_locations = df_1706.loc[:, ["DirectionRef", "VehicleLocation.Latitude", "VehicleLocation.Longitude", "DelayIncreaseSincePrevCheckin", "PrevCheckin_Latitude", "PrevCheckin_Longitude", "OriginLat", "OriginLong", "DestinationLat", "DestinationLong", "OriginName", "DestinationName"]].copy().dropna()
df_1706_delay_locations = df_1706_delay_locations.loc[df_1706_delay_locations["DelayIncreaseSincePrevCheckin"] > 0]
df_1706_delay_locations

,DirectionRef,VehicleLocation.Latitude,VehicleLocation.Longitude,DelayIncreaseSincePrevCheckin,PrevCheckin_Latitude,PrevCheckin_Longitude,OriginLat,OriginLong,DestinationLat,DestinationLong,OriginName,DestinationName
61,0,40.825428,-73.891180,1827.0,40.822621,-73.909731,40.848953,-73.936874,40.825405,-73.891319,W 179 ST/BROADWAY,W FARMS RD SOUTHERN BL
143,0,40.729402,-73.974988,132.0,40.733967,-73.977409,40.711472,-74.015762,40.738625,-73.977112,SOUTH END AV/ALBANY ST,KIPS BAY 1 AV - 26 ST via AV C
168,0,40.860776,-73.892310,268.0,40.834338,-73.850331,40.817101,-73.916031,40.877934,-73.866585,3 AV/151 ST,WILLIAMSBRIDGE GUN HILL RD via WEBSTER
183,1,40.758689,-73.991158,160.0,40.740081,-73.986118,40.737995,-73.973404,40.758770,-73.991173,WATERSIDE/WATERSIDE 1,SLCT BUS PA BUS TRMNL VIA 34 ST VIA 8 AV
194,0,40.713441,-73.961972,389.0,40.702846,-73.949824,40.690567,-73.989365,40.749702,-73.938965,BOERUM PL/SCHERMERHORN ST,LI CITY QUEENS PLAZA
...,...,...,...,...,...,...,...,...,...,...,...,...
25296,1,40.708149,-73.791541,98.0,40.771479,-73.734804,40.772629,-73.795067,40.700500,-73.807838,FRANCIS LEWIS BL/27 AV,JAMAICA LIRR STA SUTPHIN BL
25316,0,40.705351,-73.805974,36.0,40.725570,-73.792235,40.707123,-73.804382,40.701790,-73.741348,153 ST/HILLSIDE AV,SPRINGFIELD BL
25333,0,40.678072,-73.952856,309.0,40.628547,-73.947209,40.585033,-73.931328,40.709316,-73.959587,KNAPP ST/SHORE PKY,SBS WLMSBRG BRDG PLZ via NSTRND via RGRS
25396,0,40.759600,-73.823805,38.0,40.761502,-73.972934,40.758999,-73.831757,40.771767,-73.733307,ROOSEVELT AV/LAYOVER,LITTLE NECK GLENWOOD ST VIA NORTHERN


In [70]:
# The next step is to create a set of journey segments so we can see where the delays are arising
df_1706_delay_locations["Segment"] = df_1706_delay_locations["OriginName"] + " to " + df_1706_delay_locations["DestinationName"]
df_1706_delay_locations["Delay_count"] = df_1706_delay_locations.groupby("Segment")["DelayIncreaseSincePrevCheckin"].transform("count")
df_1706_delay_locations["Delay_sum"] = df_1706_delay_locations.groupby("Segment")["DelayIncreaseSincePrevCheckin"].transform("sum")
df_1706_delay_locations

,DirectionRef,VehicleLocation.Latitude,VehicleLocation.Longitude,DelayIncreaseSincePrevCheckin,PrevCheckin_Latitude,PrevCheckin_Longitude,OriginLat,OriginLong,DestinationLat,DestinationLong,OriginName,DestinationName,Segment,Delay_count,Delay_sum
61,0,40.825428,-73.891180,1827.0,40.822621,-73.909731,40.848953,-73.936874,40.825405,-73.891319,W 179 ST/BROADWAY,W FARMS RD SOUTHERN BL,W 179 ST/BROADWAY to W FARMS RD SOUTHERN BL,10,3836.0
143,0,40.729402,-73.974988,132.0,40.733967,-73.977409,40.711472,-74.015762,40.738625,-73.977112,SOUTH END AV/ALBANY ST,KIPS BAY 1 AV - 26 ST via AV C,SOUTH END AV/ALBANY ST to KIPS BAY 1 AV - 26 S...,1,132.0
168,0,40.860776,-73.892310,268.0,40.834338,-73.850331,40.817101,-73.916031,40.877934,-73.866585,3 AV/151 ST,WILLIAMSBRIDGE GUN HILL RD via WEBSTER,3 AV/151 ST to WILLIAMSBRIDGE GUN HILL RD via ...,3,482.0
183,1,40.758689,-73.991158,160.0,40.740081,-73.986118,40.737995,-73.973404,40.758770,-73.991173,WATERSIDE/WATERSIDE 1,SLCT BUS PA BUS TRMNL VIA 34 ST VIA 8 AV,WATERSIDE/WATERSIDE 1 to SLCT BUS PA BUS TRMNL...,4,460.0
194,0,40.713441,-73.961972,389.0,40.702846,-73.949824,40.690567,-73.989365,40.749702,-73.938965,BOERUM PL/SCHERMERHORN ST,LI CITY QUEENS PLAZA,BOERUM PL/SCHERMERHORN ST to LI CITY QUEENS PLAZA,3,1425.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25296,1,40.708149,-73.791541,98.0,40.771479,-73.734804,40.772629,-73.795067,40.700500,-73.807838,FRANCIS LEWIS BL/27 AV,JAMAICA LIRR STA SUTPHIN BL,FRANCIS LEWIS BL/27 AV to JAMAICA LIRR STA SUT...,2,269.0
25316,0,40.705351,-73.805974,36.0,40.725570,-73.792235,40.707123,-73.804382,40.701790,-73.741348,153 ST/HILLSIDE AV,SPRINGFIELD BL,153 ST/HILLSIDE AV to SPRINGFIELD BL,2,207.0
25333,0,40.678072,-73.952856,309.0,40.628547,-73.947209,40.585033,-73.931328,40.709316,-73.959587,KNAPP ST/SHORE PKY,SBS WLMSBRG BRDG PLZ via NSTRND via RGRS,KNAPP ST/SHORE PKY to SBS WLMSBRG BRDG PLZ via...,4,1584.0
25396,0,40.759600,-73.823805,38.0,40.761502,-73.972934,40.758999,-73.831757,40.771767,-73.733307,ROOSEVELT AV/LAYOVER,LITTLE NECK GLENWOOD ST VIA NORTHERN,ROOSEVELT AV/LAYOVER to LITTLE NECK GLENWOOD S...,5,423.0


In [71]:
# Now we should add these delay columns to a full set of segments so the list of segments includes those where no delays arose
df_1706_segments = df_1706.copy()
df_1706_segments["Segment"] = df_1706_segments["OriginName"] + " to " + df_1706_segments["DestinationName"]
df_1706_segments = df_1706_segments.merge(df_1706_delay_locations, on="Segment", how="outer", suffixes=("", "_r"))
df_1706_segments = df_1706_segments.loc[:, ["Segment", "OriginName", "DestinationName", "OriginLat", "OriginLong", "DestinationLat", "DestinationLong", "Delay_count", "Delay_sum"]].drop_duplicates()
df_1706_segments["Delay_count"] = df_1706_segments["Delay_count"].fillna(0)
df_1706_segments["Delay_sum"] = df_1706_segments["Delay_sum"].fillna(0)
df_1706_segments = df_1706_segments.sort_values(by="Delay_sum", ascending=False)
df_1706_segments

,Segment,OriginName,DestinationName,OriginLat,OriginLong,DestinationLat,DestinationLong,Delay_count,Delay_sum
94925,TEARDROP/LAYOVER to ROSEDALE LIRR STA via MERRICK,TEARDROP/LAYOVER,ROSEDALE LIRR STA via MERRICK,40.701748,-73.802399,40.666012,-73.735939,1.0,85063.0
93021,WILLIAMS AV/FLATLANDS AV to WILLIAMSBURG BRIDG...,WILLIAMS AV/FLATLANDS AV,WILLIAMSBURG BRIDGE PLAZA,40.650074,-73.891472,40.709534,-73.959549,10.0,12998.0
5892,BOSTON RD/E 180 ST to SELECT BUS JAMAICA via M...,BOSTON RD/E 180 ST,SELECT BUS JAMAICA via MAIN ST,40.842560,-73.878334,40.704933,-73.793320,13.0,9639.0
48260,41 RD/MAIN ST to RIDGEWOOD TERM,41 RD/MAIN ST,RIDGEWOOD TERM,40.757389,-73.829292,40.699699,-73.910767,10.0,8811.0
25516,AV OF THE AMERICAS/W 14 ST to HARLEM 147 ST vi...,AV OF THE AMERICAS/W 14 ST,HARLEM 147 ST via 6 AV via AMSTERDAM,40.737930,-73.996346,40.821110,-73.935898,8.0,7969.0
...,...,...,...,...,...,...,...,...,...
86641,153 ST/HILLSIDE AV to 227 ST 114 AV via LIBERTY,153 ST/HILLSIDE AV,227 ST 114 AV via LIBERTY,40.707123,-73.804382,40.702110,-73.730339,0.0,0.0
95125,KNAPP ST/SHORE PKY to SBS FLUSHING AV via NOST...,KNAPP ST/SHORE PKY,SBS FLUSHING AV via NOSTRAND via ROGERS,40.585033,-73.931328,40.698883,-73.956917,0.0,0.0
41405,WOODROW RD/VINELAND AV to E MIDTOWN 57 ST via ...,WOODROW RD/VINELAND AV,E MIDTOWN 57 ST via N JERSEY via MADISON,40.543461,-74.197227,40.759918,-73.966972,0.0,0.0
95078,EASTERN PY/UTICA AV to CANARSIE E 80 ST,EASTERN PY/UTICA AV,CANARSIE E 80 ST,40.668537,-73.931236,40.627819,-73.900467,0.0,0.0


In [72]:
# Now we create a shortlist of the top delay hotspots
top_delay_hotspots = df_1706_segments.iloc[0:10]
top_delay_hotspots

,Segment,OriginName,DestinationName,OriginLat,OriginLong,DestinationLat,DestinationLong,Delay_count,Delay_sum
94925,TEARDROP/LAYOVER to ROSEDALE LIRR STA via MERRICK,TEARDROP/LAYOVER,ROSEDALE LIRR STA via MERRICK,40.701748,-73.802399,40.666012,-73.735939,1.0,85063.0
93021,WILLIAMS AV/FLATLANDS AV to WILLIAMSBURG BRIDG...,WILLIAMS AV/FLATLANDS AV,WILLIAMSBURG BRIDGE PLAZA,40.650074,-73.891472,40.709534,-73.959549,10.0,12998.0
5892,BOSTON RD/E 180 ST to SELECT BUS JAMAICA via M...,BOSTON RD/E 180 ST,SELECT BUS JAMAICA via MAIN ST,40.842560,-73.878334,40.704933,-73.793320,13.0,9639.0
48260,41 RD/MAIN ST to RIDGEWOOD TERM,41 RD/MAIN ST,RIDGEWOOD TERM,40.757389,-73.829292,40.699699,-73.910767,10.0,8811.0
25516,AV OF THE AMERICAS/W 14 ST to HARLEM 147 ST vi...,AV OF THE AMERICAS/W 14 ST,HARLEM 147 ST via 6 AV via AMSTERDAM,40.737930,-73.996346,40.821110,-73.935898,8.0,7969.0
55628,ST CLAIRE PL/W 125 ST to LIMITED FORDHAM PLZ v...,ST CLAIRE PL/W 125 ST,LIMITED FORDHAM PLZ via 3 AV,40.817066,-73.959778,40.860989,-73.890266,7.0,7749.0
64238,ASTOR PL/3 AV to FT GEORGE 193 ST via 3 AV,ASTOR PL/3 AV,FT GEORGE 193 ST via 3 AV,40.729568,-73.990051,40.835239,-73.940201,6.0,7555.0
64244,ASTOR PL/3 AV to FT GEORGE 193 ST via 3 AV,ASTOR PL/3 AV,FT GEORGE 193 ST via 3 AV,40.729568,-73.990051,40.855667,-73.925255,6.0,7555.0
30611,CADMAN PZ W/TILLARY ST to RIDGEWOOD TERM via H...,CADMAN PZ W/TILLARY ST,RIDGEWOOD TERM via HALSEY,40.695999,-73.990944,40.699291,-73.910751,7.0,7325.0
39474,JFK AIRPORT/TERMINAL 5 AirTrain STATION to BED...,JFK AIRPORT/TERMINAL 5 AirTrain STATION,BED-STUY WOODHULL HOSP,40.647278,-73.779633,40.699776,-73.941505,5.0,7069.0


## Plotting the delay hotspots

In [73]:
# We are going to plot the delays on a map to see if there's a pattern, so we need to work out where to centre the map
lat_min = df_1706["VehicleLocation.Latitude"].min()
lat_max = df_1706["VehicleLocation.Latitude"].max()
long_min = df_1706["VehicleLocation.Longitude"].min()
long_max = df_1706["VehicleLocation.Longitude"].max()
lat_middle = lat_min + ((lat_max - lat_min) / 2)
long_middle = long_min + ((long_max - long_min) / 2)

In [75]:
# And now we plot the map

import folium
import math

map_1 = folium.Map(location=[lat_middle, long_middle], tiles='openstreetmap', zoom_start=11)

from folium import Marker
counter = 1
for i in top_delay_hotspots.index:
    popup_text = (
    "{segment}: <br>"
    "{delcount} delays totalling {delsum} hours<br>"
   ).format(segment=top_delay_hotspots.loc[i, "Segment"],
            delcount=int(top_delay_hotspots.loc[i, "Delay_count"]),
            delsum=round(top_delay_hotspots.loc[i, "Delay_sum"]/3600, 1)
            )
    Marker(
        [top_delay_hotspots.loc[i, "OriginLat"], top_delay_hotspots.loc[i, "OriginLong"]],
        popup=popup_text,
        tooltip=top_delay_hotspots.loc[i, "Segment"]+" - #"+str(counter)
    ).add_to(map_1)
    counter+=1

map_1